In [ ]:
from IPython.display import display, clear_output
import utils
import widgets
import ipywidgets as ipw
import shutil
import json
import aiida_utils
import ase
import os

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
SAMPLES_COLLECTION_OPENBIS_PATH = CONFIG["samples_collection_openbis_path"]

slabs_options = [object_key for object_key, object_info in CONFIG["objects"].items() if object_info["object_type"] == "slab_concept"]
slabs_options.insert(0, "No material")
material_selection_radio_button = utils.Radiobuttons(
    description = '', options = slabs_options, 
    disabled = False, layout = ipw.Layout(width = '300px'), 
    style = {'description_width': "100px"}
)

molecule_selector_list = widgets.MultipleSelectorWidget("molecule_concept")
molecule_selector_list_output = ipw.Output()

add_molecule_button = utils.Button(
    description = 'Add molecule', disabled = False, button_style = '', 
    tooltip = 'Add molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_molecule_button = utils.Button(
    description = 'Remove molecule', disabled = False, button_style = '', 
    tooltip = 'Remove molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_molecule_buttons_hbox = ipw.HBox([add_molecule_button, remove_molecule_button])

product_selector_list = widgets.MultipleSelectorWidget("product_concept")
product_selector_list_output = ipw.Output()

add_product_button = utils.Button(
    description = 'Add product', disabled = False, button_style = '', 
    tooltip = 'Add product', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_product_button = utils.Button(
    description = 'Remove product', disabled = False, button_style = '', 
    tooltip = 'Remove product', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_product_buttons_hbox = ipw.HBox([add_product_button, remove_product_button])

material_selector = widgets.MaterialSelectionWidget()

atomistic_model_ase_upload = ipw.FileUpload(multiple = False, accept = '.xyz')

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def add_molecule_widget(b):
    molecule_selector_list.add_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def remove_molecule_widget(b):
    molecule_selector_list.remove_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def add_product_widget(b):
    product_selector_list.add_selector()
    with product_selector_list_output:
        clear_output()
        display(product_selector_list)

def remove_product_widget(b):
    product_selector_list.remove_selector()
    with product_selector_list_output:
        clear_output()
        display(product_selector_list)

def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))
    
# Function to create atomistic model object inside openBIS using information selected in the app
def create_atomistic_model_action(b):
    atom_model_parents = []
    if material_selector.dropdown.value != -1:
        atom_model_parents.append(material_selector.dropdown.value)
    if molecule_selector_list.selectors:
        for molecule_selector in molecule_selector_list.selectors:
            if molecule_selector.dropdown.value != -1:
                atom_model_parents.append(molecule_selector.dropdown.value)
    if product_selector_list.selectors:
        for product_selector in product_selector_list.selectors:
            if product_selector.dropdown.value != -1:
                atom_model_parents.append(product_selector.dropdown.value)
    
    object_properties = {}
    is_ase_model_valid = False
    if atomistic_model_ase_upload.value:
        for filename in atomistic_model_ase_upload.value:
            file_info = atomistic_model_ase_upload.value[filename]
            utils.save_file(file_info['content'], filename)
            try:
                ase_model = ase.io.read(filename)
                is_ase_model_valid = True
            except ase.io.formats.UnknownFileTypeError:
                is_ase_model_valid = False
    
    if is_ase_model_valid:  
        dimensionality = aiida_utils.guess_dimensionality(ase_model)
        object_properties["$name"] = ase_model.get_chemical_formula()
        object_properties["structure"] = json.dumps(ase.io.jsonio.encode(ase_model))
        object_properties["dimensionality"] = int(dimensionality[0])
        object_properties["periodic_boundary_conditions"] = [bool(i) for i in dimensionality[1]]
                        
        atomistic_model_object = utils.create_openbis_object(
            OPENBIS_SESSION, type="ATOMISTIC_MODEL", 
            collection="/MATERIALS/ATOMISTIC_MODELS/ATOMISTIC_MODEL_COLLECTION", 
            props=object_properties, parents=atom_model_parents
        )
        
        utils.create_openbis_dataset(
            OPENBIS_SESSION, type = "ELN_PREVIEW", 
            sample = atomistic_model_object, files = [aiida_utils.geo_to_png(ase_model)]
        )
        
        utils.create_openbis_dataset(
            OPENBIS_SESSION, type = "RAW_DATA", 
            sample = atomistic_model_object, files = [filename]
        )
        
        os.remove(filename)
        
        display(utils.Javascript(data = "alert('Upload successful!')"))
    else:
        display(utils.Javascript(data = "alert('Upload an ASE model.')"))

def select_material_radio_change(change):
    material_type = material_selection_radio_button.value
    if material_type == "No material":
        with material_selector:
            clear_output()
            material_selector.dropdown.value = -1
            return
    
    material_selector.details_textbox.value = ''
    material_types = {}
    for object_key, object_info in CONFIG["objects"].items():
        if object_info["object_type"] == "slab_concept":
            material_types[object_key] = (object_info["openbis_object_type"], object_info["placeholder"])
            
    with material_selector:
        clear_output()
        display_list = [
            material_selector.dropdown_boxes,
            ipw.HBox([material_selector.details_textbox, material_selector.image_box])
        ]
        material_class, placeholder = material_types.get(material_type)
        material_selector.load_dropdown_box(material_class, placeholder)
        display(ipw.VBox(display_list))

# Function to handle changes in the materials dropdown
def load_material_metadata(change):
    if material_selector.dropdown.value == -1:
        material_selector.details_textbox.value = ''
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])
        return
    
    # Get material object information and dataset
    material_object = OPENBIS_SESSION.get_object(material_selector.dropdown_boxes.children[0].value)
    material_dataset = material_object.get_datasets()[0]
    
    # Get the object image preview
    if material_dataset:
        material_dataset.download(destination="images")
        material_selector.image_box.value = utils.read_file(f"images/{material_dataset.permId}/{material_dataset.file_list[0]}")
        # Erase file after downloading it
        shutil.rmtree(f"images/{material_dataset.permId}")
    else:
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])

    # Make a string with the property values of the object
    material_metadata_string = utils.get_metadata_string(OPENBIS_SESSION, material_object, "", CONFIG)
    material_selector.details_textbox.value = material_metadata_string


# Create an atomistic model object in openBIS

## Select material

In [ ]:
display(material_selection_radio_button)
display(material_selector)

## Select molecules

In [ ]:
display(molecule_selector_list_output)
with molecule_selector_list_output:
    display(molecule_selector_list)  
display(add_remove_molecule_buttons_hbox)

## Select reaction products

In [ ]:
display(product_selector_list_output)
with product_selector_list_output:
    display(product_selector_list)  
display(add_remove_product_buttons_hbox)

## Upload ASE model

In [ ]:
display(atomistic_model_ase_upload)

In [ ]:
create_button.on_click(create_atomistic_model_action)
quit_button.on_click(close_notebook)
add_molecule_button.on_click(add_molecule_widget)
remove_molecule_button.on_click(remove_molecule_widget)
add_product_button.on_click(add_product_widget)
remove_product_button.on_click(remove_product_widget)
material_selection_radio_button.observe(select_material_radio_change, names='value')
material_selector.dropdown.observe(load_material_metadata, names = 'value')
display(save_close_buttons_hbox)
display(increase_buttons_size)